In [1]:
import pandas as pd
import pickle
import matplotlib.pyplot as plt

from surprise import Dataset, Reader, accuracy , KNNWithMeans
from surprise.model_selection import cross_validate, train_test_split, GridSearchCV
from surprise.prediction_algorithms import SVD, SVDpp, NormalPredictor

plt.style.use('fast')
%matplotlib inline

In [2]:
music_review=pd.read_csv('./Data/music_review.csv')

In [3]:
music_review.isna().sum()

Unnamed: 0    0
overall       0
reviewerID    0
asin          0
reviewText    5
dtype: int64

In [4]:
music_review=music_review.drop('Unnamed: 0' , axis=1 )

In [5]:
music_review.dropna(inplace=True)

In [6]:
music_review.head()

,overall,reviewerID,asin,reviewText
0,5,A1ZCPG3D3HGRSS,0001388703,This is a great cd full of worship favorites!!...
1,5,AC2PL52NKPL29,0001388703,"So creative! Love his music - the words, the ..."
2,5,A1SUZXBDZSDQ3A,0001388703,"Keith Green, gone far to early in his carreer,..."
3,5,A3A0W7FZXM0IZW,0001388703,Keith Green had his special comedy style of Ch...
4,5,A12R54MKO17TW0,0001388703,Keith Green / So you wanna go back to Egypt......


First_simple_Modeling

NoramlPredictor

In [7]:
reader = Reader(rating_scale=(1, 5))
music_review = Dataset.load_from_df(music_review[['reviewerID', 'asin', 'overall']], reader)

trainset, testset = train_test_split(music_review, test_size=0.2, random_state=42)


In [8]:
baseline = NormalPredictor()
baseline.fit(trainset)

# Return test predictions for model fit on trainset
predictions = baseline.test(testset)

# Save RMSE score to variable
results = cross_validate(baseline, music_review, measures=['RMSE'], cv=5, verbose=True)

# Average RMSE
average_rmse = sum(results['test_rmse']) / len(results['test_rmse'])
print(f'Average RMSE using NormalPredictor: {average_rmse}')

Evaluating RMSE of algorithm NormalPredictor on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.0603  1.0599  1.0596  1.0592  1.0577  1.0593  0.0009  
Fit time          2.78    2.92    3.20    3.02    2.99    2.98    0.14    
Test time         4.17    3.98    2.64    2.56    2.67    3.20    0.71    
Average RMSE using NormalPredictor: 1.0593271695817152


baseline only

In [9]:
# Instantiate and fit model
baseline2 = BaselineOnly()
baseline2.fit(trainset)

# Return test predictions for model fit on trainset
predictions = baseline2.test(testset)

# Save RMSE score to variable
baseline_only = accuracy.rmse(predictions)

results = cross_validate(baseline, music_review, measures=['RMSE'], cv=5, verbose=True)

# Average RMSE
average_rmse = sum(results['test_rmse']) / len(results['test_rmse'])
print(f'Average RMSE using NormalPredictor: {average_rmse}')

Estimating biases using als...
RMSE: 0.7939
Evaluating RMSE of algorithm NormalPredictor on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.0568  1.0585  1.0611  1.0559  1.0613  1.0587  0.0022  
Fit time          3.12    3.10    3.17    2.92    2.88    3.04    0.12    
Test time         2.63    4.12    4.13    2.47    2.45    3.16    0.79    
Average RMSE using NormalPredictor: 1.0587030735376393


SVD

In [10]:
# Cross validate a basic SVD with no hyperparameter tuning expecting sub-par results
svd_basic = SVD(random_state=42)
results = cross_validate(svd_basic, music_review, measures=['RMSE'], cv=3, n_jobs = -1, verbose=True)

# Average RMSE
average_rmse = sum(results['test_rmse']) / len(results['test_rmse'])
print(f'Average RMSE using NormalPredictor: {average_rmse}')

Evaluating RMSE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.7934  0.7955  0.7980  0.7956  0.0019  
Fit time          91.65   91.60   92.30   91.85   0.32    
Test time         6.39    6.21    5.62    6.07    0.33    
Average RMSE using NormalPredictor: 0.7956425837128531


SVD++

In [ ]:
# Define a range of hyperparameters for GridSearchCV with SVD++
param_grid = {
    'n_factors': [50, 100, 150],  # Number of factors
    'n_epochs': [20, 30],  # Number of epochs
    'lr_all': [0.005, 0.01],  # Learning rate
    'reg_all': [0.02, 0.1]  # Regularization term
}

# Setup GridSearchCV with SVD++
svdpp = GridSearchCV(SVDpp, param_grid, measures=['rmse', 'mae'], cv=3, n_jobs=-1)

# Fit the model
gs_svdpp.fit(music_review)

# Best score and parameters
best_rmse = gs_svdpp.best_score['rmse']
best_params = gs_svdpp.best_params['rmse']

print(f"Best RMSE: {best_rmse}")
print(f"Best parameters: {best_params}")